In [3]:
import sqlite3
from tokens import TOKENVK
import pandas as pd
import requests
import datetime

con = sqlite3.connect("VK_page_testcase2.db")

In [161]:
# Парсер постов со страницы с использованием VK API
TOKEN_USER = TOKENVK
VERSION = 5.199
OWNER_ID = "ltkitten"

response = requests.get('https://api.vk.com/method/wall.get',
params={'access_token': TOKEN_USER,
        'v': VERSION,
        'count': 100,
        'owner_id': OWNER_ID,
        'filter': str('owner')})

data = response.json()['response']['items']
print(data[0])

{'inner_type': 'wall_wallpost', 'donut': {'is_donut': False}, 'comments': {'count': 0}, 'marked_as_ads': 0, 'hash': 'v87sZLbnXNlYvk3onA', 'type': 'post', 'attachments': [{'type': 'photo', 'photo': {'album_id': -7, 'date': 1688640990, 'id': 457260872, 'owner_id': 102183885, 'access_key': '3018bce535014db2e1', 'post_id': 15381, 'sizes': [{'height': 75, 'type': 's', 'width': 56, 'url': 'https://sun9-78.userapi.com/s/v1/ig2/-7E3tJJ5nyYsovyis62ToV90CGhhkej32Lq6YyWxPKeFFOeVfwvDI-vJvxNEX6jgPJdEiQGe7o2hCztJIQL_7J_0.jpg?quality=95&as=32x43,48x64,72x96,108x144,160x213,240x320,360x480,480x640,540x720,640x854,720x960,1080x1441,1280x1708,1440x1921,1919x2560&from=bu&cs=56x75'}, {'height': 130, 'type': 'm', 'width': 97, 'url': 'https://sun9-78.userapi.com/s/v1/ig2/-7E3tJJ5nyYsovyis62ToV90CGhhkej32Lq6YyWxPKeFFOeVfwvDI-vJvxNEX6jgPJdEiQGe7o2hCztJIQL_7J_0.jpg?quality=95&as=32x43,48x64,72x96,108x144,160x213,240x320,360x480,480x640,540x720,640x854,720x960,1080x1441,1280x1708,1440x1921,1919x2560&from=bu&cs=

In [162]:
# Создаем Список для дальнейшей загрузки в таблицу.
data_to_sql = []
for i in range(len(data)):
    temp_list = []
    temp_list.append(data[i]['likes']['count'])
    temp_list.append(data[i]['date'])
    data_to_sql.append(temp_list)
    data_to_sql[i][1] = datetime.datetime.fromtimestamp(data_to_sql[i][1])

In [4]:
# Создаем подключение к движку sqlite
cur = con.cursor()

In [164]:
# Создаем таблицу и выполняем запрос
cur.execute("CREATE TABLE VkPosts(likes, Date, Daypart, Weekday)")
con.commit()

In [165]:
# Создаем столбец времени суток для аналитики
for i in range(len(data_to_sql)):
    if data_to_sql[i][1].time() <= datetime.time(5, 59, 59):
        data_to_sql[i].append("Night")
        continue
    if data_to_sql[i][1].time() <= datetime.time(11, 59, 59):
        data_to_sql[i].append("Morning")
        continue
    if data_to_sql[i][1].time() <= datetime.time(17, 59, 59):
        data_to_sql[i].append("Afternoon")
        continue
    if data_to_sql[i][1].time() <= datetime.time(23, 59, 59):
        data_to_sql[i].append("Evening")
        continue

In [166]:
# Создаем столбец дня недели для аналитики
for i in range(len(data_to_sql)):
    if data_to_sql[i][1].weekday() == 0:
        data_to_sql[i].append('Monday')
        continue
    if data_to_sql[i][1].weekday() == 1:
        data_to_sql[i].append('Tuesday')
        continue
    if data_to_sql[i][1].weekday() == 2:
        data_to_sql[i].append('Wednesday')
        continue
    if data_to_sql[i][1].weekday() == 3:
        data_to_sql[i].append('Thursday')
        continue
    if data_to_sql[i][1].weekday() == 4:
        data_to_sql[i].append('Friday')
        continue
    if data_to_sql[i][1].weekday() == 5:
        data_to_sql[i].append('Saturday')
        continue
    if data_to_sql[i][1].weekday() == 6:
        data_to_sql[i].append('Sunday')
        continue
for i in range(len(data_to_sql)):
    print(data_to_sql[i])

[108, datetime.datetime(2023, 7, 6, 13, 56, 56), 'Afternoon', 'Thursday']
[51, datetime.datetime(2023, 5, 25, 14, 30, 48), 'Afternoon', 'Thursday']
[67, datetime.datetime(2023, 5, 6, 1, 1, 40), 'Night', 'Saturday']
[66, datetime.datetime(2023, 3, 7, 18, 36, 51), 'Evening', 'Tuesday']
[89, datetime.datetime(2023, 2, 8, 0, 7, 31), 'Night', 'Wednesday']
[86, datetime.datetime(2022, 10, 19, 11, 4, 44), 'Morning', 'Wednesday']
[70, datetime.datetime(2022, 8, 31, 2, 42, 41), 'Night', 'Wednesday']
[110, datetime.datetime(2022, 6, 29, 15, 42, 46), 'Afternoon', 'Wednesday']
[98, datetime.datetime(2022, 6, 25, 17, 33, 45), 'Afternoon', 'Saturday']
[87, datetime.datetime(2022, 6, 17, 20, 29, 10), 'Evening', 'Friday']
[132, datetime.datetime(2022, 6, 14, 14, 2, 12), 'Afternoon', 'Tuesday']
[103, datetime.datetime(2022, 6, 4, 17, 57, 22), 'Afternoon', 'Saturday']
[14, datetime.datetime(2022, 5, 26, 10, 44, 38), 'Morning', 'Thursday']
[60, datetime.datetime(2022, 5, 19, 13, 59, 20), 'Afternoon', 'Th

In [167]:
# Загружаем спарсенные данные в таблицу
insert_query = ("INSERT INTO VkPosts(likes, Date, Daypart, Weekday) VALUES (?, ?, ?, ?)")
cur.executemany(insert_query, data_to_sql)
con.commit()

In [59]:
cur.close()

In [172]:
# Начинаем писать сами запросы.
# Запрос 1. Влияет ли время суток на количество лайков?
cur.execute("SELECT SUM(likes), Daypart FROM VkPosts GROUP BY Daypart")
sqldata=cur.fetchall()
print(sqldata)

[(1463, 'Afternoon'), (1035, 'Evening'), (624, 'Morning'), (916, 'Night')]


In [173]:
# Запрос 2. Влияет ли день недели на количество лайков?
cur.execute("SELECT SUM(likes), Weekday FROM VkPosts WHERE likes GROUP BY Weekday")
sqldata=cur.fetchall()
print(sqldata)

[(877, 'Friday'), (226, 'Monday'), (567, 'Saturday'), (594, 'Sunday'), (425, 'Thursday'), (755, 'Tuesday'), (594, 'Wednesday')]


In [4]:
# Запрос 3. Влияет ли промежуток?
cur.execute('''SELECT
    likes,
    Date,
    JULIANDAY(Date) - JULIANDAY(LAG(Date) OVER (ORDER BY Date)) AS day_diff
FROM
    VkPosts
ORDER BY
    Date DESC;''')
sqldata=cur.fetchall()
print(sqldata)

[(108, '2023-07-06 13:56:56', 41.97648148145527), (51, '2023-05-25 14:30:48', 19.561898148152977), (67, '2023-05-06 01:01:40', 59.26723379641771), (66, '2023-03-07 18:36:51', 27.770370370242745), (89, '2023-02-08 00:07:31', 111.5435995371081), (86, '2022-10-19 11:04:44', 49.348645833320916), (70, '2022-08-31 02:42:41', 62.45827546296641), (110, '2022-06-29 15:42:46', 3.9229282406158745), (98, '2022-06-25 17:33:45', 7.878182870335877), (87, '2022-06-17 20:29:10', 3.2687268522568047), (132, '2022-06-14 14:02:12', 9.836689814459532), (103, '2022-06-04 17:57:22', 9.30050925957039), (14, '2022-05-26 10:44:38', 6.864791666623205), (60, '2022-05-19 13:59:20', 14.49914351850748), (87, '2022-05-05 02:00:34', 0.0013194442726671696), (17, '2022-05-05 01:58:40', 10.22076388914138), (86, '2022-04-24 20:40:46', 0.8226157403551042), (49, '2022-04-24 00:56:12', 5.384861111175269), (52, '2022-04-18 15:42:00', 2.8430787036195397), (70, '2022-04-15 19:27:58', 8.44406250026077), (11, '2022-04-07 08:48:31'

In [5]:
# Задание 3. Уберем выбросы в ллайках на моей странице.
cur.execute('''SELECT
    likes,
    Date,
    JULIANDAY(Date) - JULIANDAY(LAG(Date) OVER (ORDER BY Date)) AS day_diff
FROM
    VkPosts
WHERE likes < 25
ORDER BY
    Date DESC;''')
sqldata=cur.fetchall()
print(sqldata)

[(14, '2022-05-26 10:44:38', 21.365254629403353), (17, '2022-05-05 01:58:40', 27.715381944552064), (11, '2022-04-07 08:48:31', 15.773611111100763), (7, '2022-03-22 14:14:31', 192.10337962955236), (6, '2021-09-11 11:45:39', 178.2589583331719), (4, '2021-03-17 05:32:45', 2.032650463283062), (12, '2021-03-15 04:45:44', 0.0006597219035029411), (21, '2021-03-15 04:44:47', 69.18504629656672), (10, '2021-01-05 00:18:19', 103.83928240742534), (6, '2020-09-23 04:09:45', 14.206273147836328), (10, '2020-09-08 23:12:43', 6.718148148152977), (24, '2020-09-02 05:58:35', 326.5803703707643), (2, '2019-10-11 16:02:51', 71.23045138875023), (4, '2019-08-01 10:31:00', 30.930740740615875), (7, '2019-07-01 12:10:44', 165.48254629643634), (14, '2019-01-17 00:35:52', 15.291400462854654), (15, '2019-01-01 17:36:15', 62.836412037257105), (12, '2018-10-30 21:31:49', 106.80135416658595), (8, '2018-07-16 02:17:52', 10.070624999701977), (5, '2018-07-06 00:36:10', 76.21328703733161), (7, '2018-04-20 19:29:02', 6.422

In [139]:
db_df = pd.read_sql_query("SELECT * FROM VkPosts", con)
db_df.to_csv('database2.csv', index=False)

In [156]:
con.close()